In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
df=pd.read_csv("Dataset Folder/Processed_dataset.csv")
data = pd.DataFrame(df)

print(df.head())

<h2 style="color:pink; font-family:Time New Roman;">Classification:<br>
-------------------------------------------------------------------------</h2>


<h3 style="color:blue; font-family:Time New Roman;">First:</h3>

<p style="font-family:Time New Roman;">Example</p>

<h2 style="color:red;font-family:Time New Roman;">Clustering:<br>
-------------------------------------------------------------------------</h2>